<a href="https://colab.research.google.com/github/Joizra/Python-Random-Forest-Hybrid-Feature-Selection-Method-and-Random-Forest-for-Predicting-Crop-Yield/blob/main/Global_Causal_Discovery_labelencoded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install causal-learn
!pip install pgmpy
!pip install castle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

In [ ]:
# Specify the path to the 7-Zip archive file
archive_path2= '/content/drive/My Drive/Colab Notebooks/Project/mds_data.feather'
df = pd.read_feather(archive_path2)

In [ ]:
df = pd.DataFrame(data=df)

In [ ]:
location = ['emerald', 'dalby', 'taroom', 'mungindi']
df = df[df['name'].isin(location)].reset_index(drop=True)

In [ ]:
df

,name,longitude,latitude,simDoc,soilCapacity,initialSW,NFert,sowDate,cultivar,pldensity,...,RainSum,RainBeforeFlowering,RainAroundFlowering,RainAfterFlowering,PTQSum,RunOffSum,MaxTAVG,MinTAVG,MeanTAVG,CGRAroundFlowering
0,emerald,148.165108,-23.511288,148.2_-23.5,base,25top,0,15-apr,mid,mid,...,94.9,33.7,48.5,12.7,0.912,0.000,25.181,9.978,17.580,155.524
1,emerald,148.165108,-23.511288,148.2_-23.5,base,25top,0,15-apr,mid,mid,...,36.4,21.1,0.0,15.3,1.070,0.000,24.963,6.979,15.971,27.071
2,emerald,148.165108,-23.511288,148.2_-23.5,base,25top,0,15-apr,mid,mid,...,422.7,270.6,152.1,0.0,0.937,90.279,24.904,9.834,17.369,139.898
3,emerald,148.165108,-23.511288,148.2_-23.5,base,25top,0,15-apr,mid,mid,...,243.8,219.8,20.5,3.5,1.001,23.366,23.940,8.495,16.218,163.028
4,emerald,148.165108,-23.511288,148.2_-23.5,base,25top,0,15-apr,mid,mid,...,159.5,39.0,111.8,8.7,0.909,21.629,25.146,9.936,17.541,147.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842235,taroom,149.799006,-25.626871,149.8_-25.6,d,100,300,15-sep,mid,mid,...,312.8,95.0,80.0,137.8,1.225,3.473,33.361,18.477,25.919,13.731
842236,taroom,149.799006,-25.626871,149.8_-25.6,d,100,300,15-sep,mid,mid,...,217.5,74.4,78.6,64.5,1.108,0.000,32.417,17.006,24.711,122.058
842237,taroom,149.799006,-25.626871,149.8_-25.6,d,100,300,15-sep,mid,mid,...,165.2,112.7,10.7,41.8,1.267,3.892,33.022,17.910,25.466,49.390
842238,taroom,149.799006,-25.626871,149.8_-25.6,d,100,300,15-sep,mid,mid,...,314.3,135.0,41.6,137.7,1.085,4.184,32.432,18.205,25.319,233.114


In [ ]:
# Convert name from object to category
df.drop(columns=['simDoc','cultivar','pldensity', 'SSC_slowstart','longitude',	'latitude' ], inplace=True)
# df['name'] = df['name'].astype('category')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842240 entries, 0 to 842239
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   name                 842240 non-null  object  
 1   soilCapacity         842240 non-null  category
 2   initialSW            842240 non-null  category
 3   NFert                842240 non-null  category
 4   sowDate              842240 non-null  category
 5   SSC_start            842240 non-null  float64 
 6   year                 842240 non-null  int32   
 7   yield                842240 non-null  float64 
 8   pawc                 842240 non-null  float64 
 9   EpSum                842240 non-null  float64 
 10  EpBeforeFlowering    842240 non-null  float64 
 11  EpAroundFlowering    842240 non-null  float64 
 12  EpAfterFlowering     842240 non-null  float64 
 13  EsSum                842240 non-null  float64 
 14  EsBeforeFlowering    842240 non-null  float64 
 15  

In [ ]:
# Move column at position n to position 0
n = 0  # specify the index of the column to be moved, in this case, column 'D'
cols = list(df.columns)
cols = [cols[7]] + [col for col in cols if col != cols[7]]

# Reindex DataFrame with the reordered columns
df = df[cols]

In [ ]:
df

,yield,name,soilCapacity,initialSW,NFert,sowDate,SSC_start,year,pawc,EpSum,...,RainSum,RainBeforeFlowering,RainAroundFlowering,RainAfterFlowering,PTQSum,RunOffSum,MaxTAVG,MinTAVG,MeanTAVG,CGRAroundFlowering
0,1260.130,emerald,base,25top,0,15-apr,500.0,1910,60.0,87.780,...,94.9,33.7,48.5,12.7,0.912,0.000,25.181,9.978,17.580,155.524
1,483.800,emerald,base,25top,0,15-apr,500.0,1911,60.0,53.304,...,36.4,21.1,0.0,15.3,1.070,0.000,24.963,6.979,15.971,27.071
2,1216.562,emerald,base,25top,0,15-apr,500.0,1912,60.0,80.803,...,422.7,270.6,152.1,0.0,0.937,90.279,24.904,9.834,17.369,139.898
3,1045.829,emerald,base,25top,0,15-apr,500.0,1913,60.0,75.821,...,243.8,219.8,20.5,3.5,1.001,23.366,23.940,8.495,16.218,163.028
4,1291.633,emerald,base,25top,0,15-apr,500.0,1914,60.0,82.533,...,159.5,39.0,111.8,8.7,0.909,21.629,25.146,9.936,17.541,147.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842235,543.576,taroom,d,100,300,15-sep,2000.0,2015,150.0,133.698,...,312.8,95.0,80.0,137.8,1.225,3.473,33.361,18.477,25.919,13.731
842236,1445.473,taroom,d,100,300,15-sep,2000.0,2016,150.0,165.339,...,217.5,74.4,78.6,64.5,1.108,0.000,32.417,17.006,24.711,122.058
842237,745.685,taroom,d,100,300,15-sep,2000.0,2017,150.0,131.771,...,165.2,112.7,10.7,41.8,1.267,3.892,33.022,17.910,25.466,49.390
842238,2896.737,taroom,d,100,300,15-sep,2000.0,2018,150.0,214.498,...,314.3,135.0,41.6,137.7,1.085,4.184,32.432,18.205,25.319,233.114


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame
columns_to_encode = [ 'soilCapacity', 'initialSW', 'NFert', 'sowDate', 'name']

# Initialize the LabelEncoder
encoder = LabelEncoder()

# Loop through each column and update the DataFrame with encoded columns
for column in columns_to_encode:
    df[column] = encoder.fit_transform(df[column])


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842240 entries, 0 to 842239
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   yield                842240 non-null  float64
 1   name                 842240 non-null  int64  
 2   soilCapacity         842240 non-null  int64  
 3   initialSW            842240 non-null  int64  
 4   NFert                842240 non-null  int64  
 5   sowDate              842240 non-null  int64  
 6   SSC_start            842240 non-null  float64
 7   year                 842240 non-null  int32  
 8   pawc                 842240 non-null  float64
 9   EpSum                842240 non-null  float64
 10  EpBeforeFlowering    842240 non-null  float64
 11  EpAroundFlowering    842240 non-null  float64
 12  EpAfterFlowering     842240 non-null  float64
 13  EsSum                842240 non-null  float64
 14  EsBeforeFlowering    842240 non-null  float64
 15  EsAroundFlowering

# Causal Discovery

## PC

When dealing with datasets containing various data types (numerical, categorical, dates), the PC algorithm from causallearn requires numerical data. You should preprocess your dataset to convert all features into a numerical format. Here's how you can handle different data types:

Numerical Data: Keep as-is, ensuring no missing values.
Categorical Data: Convert to numerical using one-hot encoding or label encoding.
Dates: Transform into a numerical representation (e.g., Unix timestamp, extracting year/month/day as separate features).

In [ ]:
from causallearn.search.ConstraintBased.PC import pc
import pandas as pd
import numpy as np
from causallearn.utils.GraphUtils import GraphUtils

In [ ]:
data = np.array(df)
# Run PC algorithm with default independence test and alpha level

cg = pc(data)

# Save the graph to a file
pyd = GraphUtils.to_pydot(cg.G)
pyd.write_png('causal_graph.png')


  0%|          | 0/31 [00:00<?, ?it/s]